# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [1]:
resource_group = "fbaksrg" # Feel free to modify these
aks_name = "fbAKSClustergpu"
location = "eastus"

image_name = "fboylu/kerasres50-gpu" 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [6]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [3]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier.

In [6]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [3]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [5]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustergpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [6]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     25m       v1.9.6


Let's check the pods on our cluster.

In [10]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running   2          29m
kube-system   heapster-55f855b47-4m7xr                2/2       Running   0          24m
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running   0          29m
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running   0          29m
kube-system   kube-proxy-k8t2c                        1/1       Running   0          26m
kube-system   kube-svc-redirect-z6ppp                 1/1       Running   0          26m
kube-system   kubernetes-dashboard-546f987686-8krxm   1/1       Running   2          29m
kube-system   tunnelfront-695bcbdc68-t4l8t            1/1       Running   0          29m


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [11]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerasres50-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [12]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [13]:
write_json_to_file(app_template, 'az-dl.json')

In [14]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [15]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/kerasres50-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
            

Next, we will use kubectl create command to deploy our application.

In [16]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [57]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS             RESTARTS   AGE
default       azure-dl-7677f788fc-ld67s               1/1       Running            0          2h
default       azure-dl-7677f788fc-lv97p               1/1       Running            0          41m
default       azure-dl-7677f788fc-wnmff               0/1       ImagePullBackOff   0          7m
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running            2          2h
kube-system   heapster-55f855b47-4m7xr                2/2       Running            0          2h
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running            0          2h
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running            0          2h
kube-system   kube-proxy-ghjwk                        1/1       Running            0          47m
kube-system   kube-proxy-jmv6n                        1/1       Running            0          10m
kube-system   kube-proxy-k

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [56]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                    SOURCE                                 MESSAGE
53m         53m          1         aks-nodepool1-28016997-0.1536381ffc8ee781    Node                                     Normal    RegisteredNode            node-controller                        Node aks-nodepool1-28016997-0 event: Registered Node aks-nodepool1-28016997-0 in Controller
14m         14m          1         aks-nodepool1-28016997-0.15363a3d422b73c7    Node                                     Normal    RegisteredNode            node-controller                        Node aks-nodepool1-28016997-0 event: Registered Node aks-nodepool1-28016997-0 in Controller
47m         47m          1         aks-nodepool1-28016997-1.15363872946e7e16    Node                                     Normal    Starting                  kubelet, aks-nodepool1-28016997-1      Starting kubelet.
47m   

Check the logs for the application pod.

In [19]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [28]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-06-08 13:59:47,185 CRIT Supervisor running as root (no user in config file)
2018-06-08 13:59:47,188 INFO supervisord started with pid 1
2018-06-08 13:59:48,190 INFO spawned: 'program_exit' with pid 10
2018-06-08 13:59:48,192 INFO spawned: 'nginx' with pid 11
2018-06-08 13:59:48,193 INFO spawned: 'gunicorn' with pid 12
2018-06-08 13:59:49,226 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-08 13:59:49.604948: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-08 13:59:49.796415: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: ddde:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-08 13:59:49.796461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1312] Adding visible gpu devices: 0
2018-06-08

In [50]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   2         2         2            2           1h


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [36]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)        AGE
azure-dl   LoadBalancer   10.0.150.121   40.121.69.40   80:31965/TCP   18m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Scaling

In [51]:
!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 3

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 3,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

In [52]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     2h        v1.9.6
aks-nodepool1-28016997-1   Ready     agent     40m       v1.9.6
aks-nodepool1-28016997-2   Ready     agent     3m        v1.9.6


In [53]:
!kubectl scale --current-replicas=2 --replicas=3 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [59]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-7677f788fc-ld67s               1/1       Running   0          2h
default       azure-dl-7677f788fc-lv97p               1/1       Running   0          44m
default       azure-dl-7677f788fc-wnmff               1/1       Running   0          10m
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running   2          2h
kube-system   heapster-55f855b47-4m7xr                2/2       Running   0          2h
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running   0          2h
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running   0          2h
kube-system   kube-proxy-ghjwk                        1/1       Running   0          51m
kube-system   kube-proxy-jmv6n                        1/1       Running   0          14m
kube-system   kube-proxy-k8t2c                        1/1       Running   0          2h
kube-system   kube-svc-redi

In [61]:
!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 1

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [60]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

In [4]:
!az group delete --name $resource_group -y

 - Finished ..